In [1]:
import sys
print(sys.version)
username = 'admin' # 'lucag' #'admin' # 'admin' for BO1 lab
sys.path.append('c:/users/' + username + '/picosdk-python-wrappers')
sys.path.append('c:/Users/lucag/anaconda3/Lib/site-packages/picosdk-1.0/picosdk-1.0/picosdk') # not for BO1 lab
path_to_MyCustomPackage = '../' # relative path to the MyCustomPackage folder 
sys.path.append(path_to_MyCustomPackage)
from MyCustomPackage import mycustommodule
import picosdk
import os
os.getcwd()
import logging
import ctypes
import numpy as np #
from picosdk.ps4000 import ps4000 as ps #
import matplotlib.pyplot as plt
from picosdk.functions import adc2mV, assert_pico_ok
from statistics import mean
import time
import math

%matplotlib inline

def round_half_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n*multiplier + 0.5) / multiplier

3.9.5 (default, May 18 2021, 14:42:02) [MSC v.1916 64 bit (AMD64)]


In [2]:
# Starting Pico with range of 2V, sample interval of 250 microseconds and 5000 total samples

channel_range = ps.PS4000_RANGE['PS4000_2V']
sampleInterval = ctypes.c_int32(250) # in sample units specified by sampleUnits
sampleUnits = ps.PS4000_TIME_UNITS['PS4000_US']
sizeOfOneBuffer = 500 # size of a single buffer
numBuffersToCapture = 10 # totalSamples = sizeOfOneBuffer * numBuffersToCapture
mycustommodule.pico_start(channel_range, sampleInterval = sampleInterval, sampleUnits = sampleUnits, sizeOfOneBuffer = sizeOfOneBuffer, numBuffersToCapture = numBuffersToCapture)

# We need a big buffer, not registered with the driver, to keep our complete capture in.
for i in range(4):

    print(mycustommodule.pico_acquire_measurement(channel_range, sampleInterval = sampleInterval, sampleUnits = sampleUnits, sizeOfOneBuffer = sizeOfOneBuffer, numBuffersToCapture = numBuffersToCapture));
    time.sleep(0.1)

mycustommodule.pico_stop()

INFO:root:Capturing at sample interval  2.500E+05 ns, with total sampling time of  1.250E+09 ns
INFO:root:Done grabbing values.


-0.914


INFO:root:Done grabbing values.


-0.952


INFO:root:Done grabbing values.


-0.966


INFO:root:Done grabbing values.


-0.985
